# Progetto d'esame di Data Analysis in Experimental Physics with Machine Learning

Gruppo composto dagli studenti Luca Attinà, Sharis Feriotto e Matteo Marchisio Caprioglio

Dataset ipotesi: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset

In [1]:
%conda install -c conda-forge kagglehub


Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /Users/2mc/miniconda3/envs/daml

  added / updated specs:
    - kagglehub


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.7.9   |       hbd8a1cb_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         149 KB

The following packages will be UPDATED:

  ca-certificates                      2025.6.15-hbd8a1cb_0 --> 2025.7.9-hbd8a1cb_0 



                                                                                
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Prova import dataset
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)


Path to dataset files: /Users/2mc/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2


In [ ]:
train_path = path + "/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
val_path = path + "/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"
test_path = path + "/test"


In [ ]:
train_path


'/Users/2mc/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

In [ ]:
%ls "{train_path}"
%ls "{val_path}"


Apple___Apple_scab/
Apple___Black_rot/
Apple___Cedar_apple_rust/
Apple___healthy/
Blueberry___healthy/
Cherry_(including_sour)___healthy/
Cherry_(including_sour)___Powdery_mildew/
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot/
Corn_(maize)___Common_rust_/
Corn_(maize)___healthy/
Corn_(maize)___Northern_Leaf_Blight/
Grape___Black_rot/
Grape___Esca_(Black_Measles)/
Grape___healthy/
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)/
Orange___Haunglongbing_(Citrus_greening)/
Peach___Bacterial_spot/
Peach___healthy/
Pepper,_bell___Bacterial_spot/
Pepper,_bell___healthy/
Potato___Early_blight/
Potato___healthy/
Potato___Late_blight/
Raspberry___healthy/
Soybean___healthy/
Squash___Powdery_mildew/
Strawberry___healthy/
Strawberry___Leaf_scorch/
Tomato___Bacterial_spot/
Tomato___Early_blight/
Tomato___healthy/
Tomato___Late_blight/
Tomato___Leaf_Mold/
Tomato___Septoria_leaf_spot/
Tomato___Spider_mites Two-spotted_spider_mite/
Tomato___Target_Spot/
Tomato___Tomato_mosaic_virus/
Tomato___Tomato_Ye

In [ ]:
import tensorflow as tf


In [ ]:
img_size = (256, 256)
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

test_ds


Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.
Found 33 files belonging to 1 classes.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>